In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
# Load the data
data = np.load('/Users/butovensmede/Downloads/data_train.npy')

# # Load the data
# train_data = np.load('/Users/butovensmede/Downloads/data_train.npy')
# test_data = np.load('/Users/butovensmede/Downloads/data_test.npy')

In [3]:
# Prepare the data
X = torch.tensor(data[:, :, :]).float()  # Features
y = torch.tensor(np.concatenate((np.zeros(87), np.ones(79)))).float()  # Labels (0 for AD, 1 for CN)

# # Prepare the data
# X_train = torch.tensor(train_data[:, :, :]).float()  # Training features
# y_train = torch.tensor(np.concatenate((np.zeros(87), np.ones(79)))).float()  # Training labels
# X_test = torch.tensor(test_data[:, :, :]).float()  # Testing features
# y_test = torch.tensor(np.concatenate((np.zeros(87), np.ones(79)))).float()  # Testing labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [37]:
# Define the RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob=0.5):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, (h0, c0))
        out = self.dropout(out)
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

In [48]:
# Initialize the model
input_size = X.shape[2]
hidden_size = 64
num_layers = 3
output_size = 1
dropout_prob = 0.2

model = RNN(input_size, hidden_size, num_layers, output_size, dropout_prob)

In [51]:
# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [27]:
# Prepare DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [52]:
# Train the model
num_epochs = 20
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/20], Loss: 0.6962
Epoch [2/20], Loss: 0.6955
Epoch [3/20], Loss: 0.7328
Epoch [4/20], Loss: 0.6983
Epoch [5/20], Loss: 0.6842
Epoch [6/20], Loss: 0.5987
Epoch [7/20], Loss: 0.7243
Epoch [8/20], Loss: 0.6817
Epoch [9/20], Loss: 0.7600
Epoch [10/20], Loss: 0.7054
Epoch [11/20], Loss: 0.7164
Epoch [12/20], Loss: 0.6692
Epoch [13/20], Loss: 0.6953
Epoch [14/20], Loss: 0.7341
Epoch [15/20], Loss: 0.7003
Epoch [16/20], Loss: 0.6930
Epoch [17/20], Loss: 0.5834
Epoch [18/20], Loss: 0.6943
Epoch [19/20], Loss: 0.6974
Epoch [20/20], Loss: 0.6879


In [53]:
# Evaluate the model
with torch.no_grad():
    outputs = model(X_test)
    predicted = (outputs.squeeze() > 0.5).float()
    accuracy = (predicted == y_test).float().mean()
    print(f'Test Accuracy: {accuracy.item():.4f}')

Test Accuracy: 0.3824
